In [1]:
import pandas as pd
import string
import random
from pandasql import sqldf

In [2]:
# Function to generate dummy data for teams
def create_teams_tbl(num_teams, team_name_len=7):
    names = {} # Initialize dictionary
    
    """
    Generate random string for team names.
    Range value started in 1 to satisfy the condition the first team must have ID = 1
    """
    for id in range(1, num_teams + 1):
        string_names = [] # Array for checking the existing random string names
        
        random_str = ''.join(random.choices(string.ascii_letters, k=team_name_len))
        
        #Check random string exists to ensure unique names
        if random_str not in string_names:
            team_name = f"Team {random_str}" # Combine Team and the random string
            string_names.append(random_str)
        
        names[id] = team_name
        
    return names

In [3]:
# Create number of teams
n_teams = create_teams_tbl(20)

In [4]:
# Show data in n_teams
n_teams

{1: 'Team ZFYRdfl',
 2: 'Team TTNLOjb',
 3: 'Team clXHhqJ',
 4: 'Team PuLTlMy',
 5: 'Team IozDjEH',
 6: 'Team xxCYTZJ',
 7: 'Team MNhUGKJ',
 8: 'Team zeNgyHB',
 9: 'Team gLDKnRu',
 10: 'Team lHgDvZk',
 11: 'Team OLXVBhy',
 12: 'Team YwJmoRM',
 13: 'Team GrqBIOA',
 14: 'Team nzKEuQx',
 15: 'Team MBDVTIo',
 16: 'Team NxbxepJ',
 17: 'Team avkZKsF',
 18: 'Team hTrTWxw',
 19: 'Team yXCeCXj',
 20: 'Team XpiFjKz'}

In [5]:
"""
From the generated IDs and team names, specifically extract all the IDs and team names.
The IDs and team names will be inserted into their respective arrays in order to align the format with the DataFrame.
"""
ids = [] # Array for the generated IDs
team_names = [] # Array for the team names

for k,v in n_teams.items():
    ids.append(k)
    team_names.append(v)

In [6]:
# Create a dictionary in the desired format
data = {
    'id': ids,
    'name': team_names
}

In [7]:
# Convert dictionary into DataFrame
df = pd.DataFrame(data)

In [8]:
# Show the DataFrame
print(df.to_string(index=False)) 

 id         name
  1 Team ZFYRdfl
  2 Team TTNLOjb
  3 Team clXHhqJ
  4 Team PuLTlMy
  5 Team IozDjEH
  6 Team xxCYTZJ
  7 Team MNhUGKJ
  8 Team zeNgyHB
  9 Team gLDKnRu
 10 Team lHgDvZk
 11 Team OLXVBhy
 12 Team YwJmoRM
 13 Team GrqBIOA
 14 Team nzKEuQx
 15 Team MBDVTIo
 16 Team NxbxepJ
 17 Team avkZKsF
 18 Team hTrTWxw
 19 Team yXCeCXj
 20 Team XpiFjKz


In [9]:
# Output table for the team names into a csv file
df.to_csv('./data/teams.csv', index=False)

In [10]:
# Check the number of key-value pairs in the dictionary
print(len(n_teams))

20


In [11]:
# Function to generate dummy data for matches
def create_matches_tbl(team_names, num_matches):
    unique_pairs = [] # List for  unique pairs of team to match atleast once
    matches = [] # Array for all the matches
    
    # For cases that cannot generate matches
    if num_matches <= 0 or len(team_names) < 2:
        return []
    
    for i1, t1 in team_names.items():
        for i2, t2 in team_names.items():
            if i1 != i2:
                if (t1, t2) not in unique_pairs and (t2, t1) not in unique_pairs:
                    unique_pairs.append((t1,t2)) # Add team1 and team2 to the unique_pairs list
    
    random.shuffle(unique_pairs) # Shuffle to avoid predictable order and randomize matchup
    
    if num_matches <= len(unique_pairs):
        matches = unique_pairs[:num_matches]
    else:
        matches = unique_pairs.copy()
        rem_matches = num_matches - len(unique_pairs)
        while rem_matches > 0:
            t1, t2 = random.sample(list(team_names.values()), 2)
            matches.append((t1, t2))
            rem_matches -= 1
            
    results = [] # Array for the result
    for match in matches:
        t1, t2 = match
        winner = random.choice([t1, t2]) # Randomly select the winner
        loser = t2 if winner == t1 else t1 # Set the loser
        results.append({"match": f"{t1} vs {t2}", "winner": winner, "loser": loser})
    
    # Create DataFrame from the result
    df = pd.DataFrame(results)
    
    # Return DataFrame
    return df

In [12]:
"""
Create matches given the team names generated after executing create_teams_tbl().
Return DataFrame as the result
"""
show_result = create_matches_tbl(n_teams, 200)

In [13]:
# Show the overview of the result
print(show_result)

                            match        winner         loser
0    Team GrqBIOA vs Team XpiFjKz  Team GrqBIOA  Team XpiFjKz
1    Team ZFYRdfl vs Team zeNgyHB  Team ZFYRdfl  Team zeNgyHB
2    Team ZFYRdfl vs Team IozDjEH  Team IozDjEH  Team ZFYRdfl
3    Team xxCYTZJ vs Team NxbxepJ  Team NxbxepJ  Team xxCYTZJ
4    Team gLDKnRu vs Team YwJmoRM  Team YwJmoRM  Team gLDKnRu
..                            ...           ...           ...
195  Team clXHhqJ vs Team hTrTWxw  Team hTrTWxw  Team clXHhqJ
196  Team nzKEuQx vs Team NxbxepJ  Team nzKEuQx  Team NxbxepJ
197  Team PuLTlMy vs Team IozDjEH  Team IozDjEH  Team PuLTlMy
198  Team hTrTWxw vs Team OLXVBhy  Team hTrTWxw  Team OLXVBhy
199  Team nzKEuQx vs Team NxbxepJ  Team nzKEuQx  Team NxbxepJ

[200 rows x 3 columns]


In [14]:
# Output table for the matches  into a csv file
show_result.to_csv('./data/matches.csv', index=False)

In [15]:
# Define SQL query for top 10 teams and their total number of wins
query = """
SELECT
    winner AS team,
    COUNT(winner) AS win_count
FROM show_result
GROUP BY winner
ORDER BY win_count DESC
LIMIT 10;
"""

In [16]:
# Execute query for the result
top_teams = sqldf(query, locals())

In [17]:
# Show top 10 teams by wins for the 2023 season
print(top_teams)

           team  win_count
0  Team hTrTWxw         14
1  Team ZFYRdfl         13
2  Team IozDjEH         13
3  Team zeNgyHB         12
4  Team xxCYTZJ         12
5  Team NxbxepJ         12
6  Team clXHhqJ         11
7  Team PuLTlMy         11
8  Team yXCeCXj         10
9  Team YwJmoRM         10


In [18]:
# Output table for the top teams into a csv file
top_teams.to_csv('./data/top_teams.csv', index=False)